In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, recall_score

import xgboost
from xgboost import XGBClassifier

# Common Tools
from sklearn.preprocessing import LabelEncoder
from collections import Counter

#Algorithms
from sklearn import ensemble, tree, svm, naive_bayes, neighbors, linear_model, gaussian_process, neural_network
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# Model
from sklearn.metrics import accuracy_score, f1_score, auc, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score

In [2]:
test_ind = pd.read_csv('/content/drive/MyDrive/BU Capstone/Results/Before midterm/Final sample design results/id_Xtest.csv').drop('Unnamed: 0', axis=1)
down_ind = pd.read_csv('/content/drive/MyDrive/BU Capstone/Results/Before midterm/Final sample design results/id_downsample.csv').drop('Unnamed: 0', axis=1)

In [3]:
id_var = pd.read_csv('/content/drive/MyDrive/BU Capstone/Results/id_filled_full.csv')

In [ ]:
cols = id_var.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
cols

In [17]:
df = id_var[cols]

In [18]:
df.head()

,pol_id,year,var1222,var1222_indicator,var0989,var0989_indicator,var0202,var0202_indicator,var1165,var1165_indicator,var0063,var0063_indicator,var0797,var0797_indicator,var0967,var0967_indicator,var0403,var0403_indicator,var0222,var0222_indicator,var0424,var0424_indicator,var0371,var0371_indicator,var0484,var0484_indicator,var0547,var0547_indicator,var0065,var0065_indicator,var0975,var0975_indicator,var0457,var0457_indicator,var0233,var0233_indicator,var0260,var0260_indicator,var0365,var0365_indicator,...,var1257,var1257_indicator,var0525,var0525_indicator,var0624,var0624_indicator,var0775,var0775_indicator,var0636,var0636_indicator,var0640,var0640_indicator,var0992,var0992_indicator,var0867,var0867_indicator,var1234,var1234_indicator,var0921,var0921_indicator,var0350,var0350_indicator,var0535,var0535_indicator,var1146,var1146_indicator,var0100,var0100_indicator,var1145,var1145_indicator,var1171,var1171_indicator,var0505,var0505_indicator,var0123,var0123_indicator,var1173,var1173_indicator,var1204,var1204_indicator
0,MWH00001004972,2016,0.023866,1,1.267882,0,0.898809,1,0.0,0,1962.0,1,0.975795,1,1.301259,0,0.483601,0,1.168248,1,0.543898,0,0.76,1,1.02,0,0.0,1,0.587383,1,0.893523,0,1.000023,1,0.92,1,1.01,1,0.85,1,...,0.0,0,164007.83,0,1.006300,1,0.668959,1,1.004272,1,0.048890,1,0.006981,0,323.0,0,0.0,0,264.0,0,0.0,1,0.0,0,0.0,0,0.0,1,0.0,0,0.0,0,0.0,0,1.0,1,0.0,0,0.0,0
1,MWH00001004972,2017,0.023866,1,1.269879,0,0.898809,1,1.0,1,1962.0,1,0.975795,1,1.299627,0,0.483601,0,1.168248,1,0.543898,0,0.76,1,1.02,0,0.0,1,0.587383,1,0.923915,0,1.000023,1,0.92,1,1.01,1,0.85,1,...,0.0,0,164007.83,0,1.065094,0,0.668959,1,1.049301,0,0.049230,0,0.006614,0,335.0,0,0.0,0,273.0,0,0.0,1,0.0,0,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0,1.0,1,0.0,1,0.0,0
2,MWH00001004972,2018,0.023866,1,1.240246,0,0.898809,1,0.0,0,1962.0,1,0.888889,0,1.297465,0,0.483601,0,1.168248,1,0.543898,0,0.76,1,1.02,0,0.0,1,0.587383,1,1.042641,0,1.000023,1,0.92,1,1.01,1,0.85,1,...,0.0,0,164007.83,0,1.065094,0,0.249928,0,1.049301,0,0.049230,0,0.007011,0,312.0,0,0.0,0,277.0,0,0.0,1,0.0,0,0.0,0,0.0,1,0.0,0,0.0,0,0.0,0,1.0,1,0.0,0,0.0,0
3,MWH00001004972,2019,0.023866,1,1.242967,0,0.898809,1,1.0,1,1962.0,1,1.000000,0,1.317773,0,0.483601,0,1.168248,1,0.543898,0,0.76,1,1.02,0,0.0,1,0.587383,1,1.234257,0,1.000023,1,0.92,1,1.01,1,0.85,1,...,0.0,0,164007.83,0,1.065286,0,0.283261,0,1.049401,0,0.045804,0,0.006559,0,311.0,0,0.0,0,295.0,0,0.0,1,0.0,0,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0,1.0,1,0.0,1,0.0,0
4,BHH00001005147,2016,0.052876,0,1.003142,0,0.898809,1,1.0,1,1962.0,1,0.975795,1,0.976341,0,0.503694,0,1.168248,1,1.399414,0,0.76,1,1.06,0,0.0,1,0.587383,1,0.857632,0,1.000023,1,0.92,1,1.01,1,0.85,1,...,152.0,0,151950.20,0,1.006300,1,0.668959,1,1.004272,1,0.048890,1,0.004129,0,56.0,0,34.0,0,171.0,0,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,1.0,1,0.0,1,0.0,1


In [19]:
df.to_csv('id_filled_full.csv', index=False)

In [5]:
imp = pd.read_csv('/content/drive/MyDrive/BU Capstone/Results/Second round model train/importance.csv')

In [10]:
imp_var = id_var.columns.to_list()
type(imp_var)

list

In [20]:
imp['in'] = imp['Unnamed: 0.1'].isin(imp_var)

In [17]:
for i in range(len(imp)):
  if imp['in'][i]==False:
    imp.drop(i, inplace=True)

In [19]:
imp.to_csv('imp_var.csv')

# fillin df

In [4]:
train_df = pd.merge(down_ind, id_var, how='inner', on=['pol_id','year'])

In [5]:
train_df.head()

,pol_id,year,STATE,ZIP5,ZIP4,y,var1222,var0989,var0202,var1165,var0063,var0797,var0967,var0403,var0222,var0424,var0371,var0484,var0547,var0065,var0975,var0457,var0233,var0260,var0365,var0359,var0188,var1022,var0335,var0462,var0071,var0499,var1025,var0176,var0294,var0391,var0140,var0393,var0372,var0980,...,var0450,var0571,var0617,var0642,var0887,var0810,var0630,var0641,var0684,var0453,var0538,var1262,var1248,var0408,var1220,var1207,var0623,var0018,var0785,var0758,var1257,var0525,var0624,var0775,var0636,var0640,var0992,var0867,var1234,var0921,var0350,var0535,var1146,var0100,var1145,var1171,var0505,var0123,var1173,var1204
0,PNH00002018078,2019,PA,19390,8825.0,0,0.005209,1.075002,0.480738,0.0,1980.0,1.000000,1.105132,0.486821,1.569459,1.052856,1.14,0.74,1.19,1.347365,0.686222,1.059349,0.18,1.01,1.66,1.53,2004.0,1.105364,3987.11,0.26,2.641293,0.24,1.025243,1.690133,2.06,1.00,1.292820,2658.07,449.00,0.780653,...,1.550902,2352.0,0.754774,0.000114,147.0,25.0,1.195731,0.074851,1.067409,0.276278,0.09,264.0,2000.0,1.302859,1398.0,1420.916667,494.1,0.000000,2581000.0,309.672222,54.0,134307.68,1.096042,0.438007,1.050029,0.046964,0.001575,59.0,3.0,140.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,BHH10001227349,2016,MA,1982,1760.0,0,0.052141,1.155330,1.363285,NaN,1936.0,NaN,1.130570,0.478463,NaN,1.091830,NaN,0.99,NaN,0.651272,1.173166,NaN,NaN,1.11,0.42,NaN,NaN,1.278215,932.00,0.07,NaN,0.30,1.117199,0.586105,NaN,0.45,NaN,1864.00,1198.00,1.153954,...,1.093688,NaN,NaN,NaN,183.0,25.0,NaN,NaN,NaN,0.235412,0.00,359.0,2276.0,1.179300,2971.0,1520.333333,NaN,3.037253,9826300.0,NaN,147.0,233126.06,NaN,NaN,NaN,NaN,0.002062,166.0,40.0,198.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,0.0
2,BHH00001046369,2016,MA,1949,1515.0,0,0.052141,0.954347,0.973775,NaN,1965.0,NaN,1.053868,0.435080,1.694850,1.333418,1.07,0.98,NaN,NaN,0.947003,NaN,0.60,1.11,1.59,0.45,1996.0,1.250351,5258.00,0.06,NaN,0.25,1.072705,NaN,NaN,1.22,NaN,5258.00,635.50,0.999961,...,1.265296,NaN,NaN,NaN,212.0,13.0,NaN,NaN,NaN,0.229599,0.00,359.0,2276.0,1.959466,2314.0,1527.041667,NaN,NaN,7176000.0,NaN,147.0,214444.13,NaN,NaN,NaN,NaN,0.001874,105.0,40.0,198.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN
3,PNH00002013737,2019,PA,19128,2504.0,0,-0.001025,1.371830,0.801230,0.0,1977.0,1.000000,1.461411,0.592008,1.856878,0.806825,1.74,0.66,0.00,0.790412,1.545858,1.058247,1.39,1.01,NaN,NaN,1950.0,1.678844,1160.44,0.39,NaN,0.25,1.241813,0.946475,0.95,NaN,NaN,2320.87,202.67,1.463356,...,1.391907,4408.0,0.735811,0.000105,40.0,13.0,1.234655,0.073477,1.103124,0.287844,0.12,265.0,1979.0,0.582472,1352.0,997.083333,6031.5,2.576455,8870000.0,54.000688,54.0,146033.17,1.140664,NaN,1.074996,0.047935,0.002056,23.0,2.0,89.0,0.0,1.43,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,PAH00001301443,2018,NJ,7041,1556.0,0,0.012427,1.198941,1.835766,NaN,1985.0,0.888889,1.186876,0.553058,1.574130,0.894913,2.57,0.97,NaN,1.298606,0.925188,1.006820,0.96,1.00,1.99,1.33,1965.0,0.587645,13694.40,0.00,1.809543,0.50,1.139927,1.555997,1.54,1.87,2.622966,NaN,NaN,0.966776,...,1.102110,1668.0,0.949219,0.000104,134.0,13.0,1.007243,0.067098,1.005579,0.234561,0.02,270.0,1887.0,0.915569,1412.0,1984.625000,5522.6,0.000000,59450200.0,70.701852,89.0,NaN,1.009578,NaN,1.009175,0.048074,0.000992,74.0,6.0,180.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN


In [6]:
# create missing value indicator
colist = list(train_df.columns)
df = train_df[colist].isnull().astype(int).add_suffix('_indicator')
cols = [item for x in colist for item in (x, x + '_indicator')]

In [7]:
var = train_df.drop(['pol_id','year','STATE','ZIP5','ZIP4'], axis=1)
var1 = var.apply(lambda x: x.fillna(x.median()))
df1 = pd.concat([var1, df], axis=1).reindex(cols, axis=1)
df1['pol_id'] = train_df['pol_id']
df1['year'] = train_df['year']
df1['STATE'] = train_df['STATE']
df1['ZIP5'] = train_df['ZIP5']
df1 = df1.drop(['pol_id_indicator','year_indicator','STATE_indicator','ZIP5_indicator'], axis=1)

In [8]:
df1 = df1.drop(['ZIP4','ZIP4_indicator','y_indicator'], axis=1)

In [9]:
df1.head()

,pol_id,year,STATE,ZIP5,y,var1222,var1222_indicator,var0989,var0989_indicator,var0202,var0202_indicator,var1165,var1165_indicator,var0063,var0063_indicator,var0797,var0797_indicator,var0967,var0967_indicator,var0403,var0403_indicator,var0222,var0222_indicator,var0424,var0424_indicator,var0371,var0371_indicator,var0484,var0484_indicator,var0547,var0547_indicator,var0065,var0065_indicator,var0975,var0975_indicator,var0457,var0457_indicator,var0233,var0233_indicator,var0260,...,var1257,var1257_indicator,var0525,var0525_indicator,var0624,var0624_indicator,var0775,var0775_indicator,var0636,var0636_indicator,var0640,var0640_indicator,var0992,var0992_indicator,var0867,var0867_indicator,var1234,var1234_indicator,var0921,var0921_indicator,var0350,var0350_indicator,var0535,var0535_indicator,var1146,var1146_indicator,var0100,var0100_indicator,var1145,var1145_indicator,var1171,var1171_indicator,var0505,var0505_indicator,var0123,var0123_indicator,var1173,var1173_indicator,var1204,var1204_indicator
0,PNH00002018078,2019,PA,19390,0,0.005209,0,1.075002,0,0.480738,0,0.0,0,1980.0,0,1.000000,0,1.105132,0,0.486821,0,1.569459,0,1.052856,0,1.14,0,0.74,0,1.19,0,1.347365,0,0.686222,0,1.059349,0,0.18,0,1.01,...,54.0,0,134307.68,0,1.096042,0,0.438007,0,1.050029,0,0.046964,0,0.001575,0,59.0,0,3.0,0,140.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,1.0,0,0.0,0,0.0,0
1,BHH10001227349,2016,MA,1982,0,0.052141,0,1.155330,0,1.363285,0,0.0,1,1936.0,0,0.977025,1,1.130570,0,0.478463,0,1.246479,1,1.091830,0,0.78,1,0.99,0,0.00,1,0.651272,0,1.173166,0,1.006814,1,0.86,1,1.11,...,147.0,0,233126.06,0,1.007284,1,0.645818,1,1.005294,1,0.049101,1,0.002062,0,166.0,0,40.0,0,198.0,0,0.0,0,0.00,1,0.0,1,1.0,0,0.0,1,0.0,1,0.0,1,1.0,0,0.0,1,0.0,0
2,BHH00001046369,2016,MA,1949,0,0.052141,0,0.954347,0,0.973775,0,0.0,1,1965.0,0,0.977025,1,1.053868,0,0.435080,0,1.694850,0,1.333418,0,1.07,0,0.98,0,0.00,1,0.640189,1,0.947003,0,1.006814,1,0.60,0,1.11,...,147.0,0,214444.13,0,1.007284,1,0.645818,1,1.005294,1,0.049101,1,0.001874,0,105.0,0,40.0,0,198.0,0,0.0,0,0.00,1,0.0,1,0.0,0,0.0,1,0.0,1,0.0,1,1.0,0,0.0,1,0.0,1
3,PNH00002013737,2019,PA,19128,0,-0.001025,0,1.371830,0,0.801230,0,0.0,0,1977.0,0,1.000000,0,1.461411,0,0.592008,0,1.856878,0,0.806825,0,1.74,0,0.66,0,0.00,0,0.790412,0,1.545858,0,1.058247,0,1.39,0,1.01,...,54.0,0,146033.17,0,1.140664,0,0.645818,1,1.074996,0,0.047935,0,0.002056,0,23.0,0,2.0,0,89.0,0,0.0,0,1.43,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,1.0,0,0.0,0,0.0,0
4,PAH00001301443,2018,NJ,7041,0,0.012427,0,1.198941,0,1.835766,0,0.0,1,1985.0,0,0.888889,0,1.186876,0,0.553058,0,1.574130,0,0.894913,0,2.57,0,0.97,0,0.00,1,1.298606,0,0.925188,0,1.006820,0,0.96,0,1.00,...,89.0,0,140060.24,1,1.009578,0,0.645818,1,1.009175,0,0.048074,0,0.000992,0,74.0,0,6.0,0,180.0,0,0.0,0,0.00,1,0.0,1,0.0,0,0.0,1,0.0,1,0.0,1,1.0,0,0.0,1,0.0,1


In [10]:
df1.to_csv('df1.csv')

In [11]:
test_df = pd.merge(test_ind, id_var, how='inner', on=['pol_id','year'])
test_df.head()

,pol_id,year,STATE,ZIP5,ZIP4,y,var1222,var0989,var0202,var1165,var0063,var0797,var0967,var0403,var0222,var0424,var0371,var0484,var0547,var0065,var0975,var0457,var0233,var0260,var0365,var0359,var0188,var1022,var0335,var0462,var0071,var0499,var1025,var0176,var0294,var0391,var0140,var0393,var0372,var0980,...,var0450,var0571,var0617,var0642,var0887,var0810,var0630,var0641,var0684,var0453,var0538,var1262,var1248,var0408,var1220,var1207,var0623,var0018,var0785,var0758,var1257,var0525,var0624,var0775,var0636,var0640,var0992,var0867,var1234,var0921,var0350,var0535,var1146,var0100,var1145,var1171,var0505,var0123,var1173,var1204
0,PNH00002016383,2019,PA,19380,1336.0,0,0.006070,1.240923,0.480738,4.0,1958.0,0.903663,1.247416,0.485741,0.000000,1.121520,0.71,0.64,1.19,1.308630,0.971220,1.054577,2.38,1.01,1.12,0.51,1962.0,1.300999,2221.3,0.30,0.660323,0.26,1.184767,1.971822,0.86,1.01,0.725058,2221.30,337.75,0.971544,...,1.582192,12209.0,0.925303,0.000125,145.0,15.0,1.267870,0.071166,1.114045,0.267241,0.10,266.0,2004.0,1.190996,778.0,1116.833333,1529.3,2.576455,35087900.0,355.162982,54.0,170919.10,1.153634,0.637127,1.083459,0.045213,0.005820,85.0,3.0,173.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,1.0,4.0,0.0
1,PNH00002005641,2018,PA,19149,1933.0,0,0.022148,1.348257,0.587569,NaN,1984.0,0.000000,1.229200,0.634735,2.288006,0.896893,1.05,0.46,NaN,0.606157,1.407050,1.042516,1.63,1.01,NaN,NaN,1950.0,2.303053,741.9,0.41,NaN,0.14,1.104377,0.619716,0.55,NaN,NaN,1483.79,195.00,1.422006,...,1.165139,379.0,0.069586,0.000080,49.0,9.0,1.159113,0.075491,1.088758,0.275072,0.13,190.0,1836.0,0.874940,1158.0,1632.041667,23153.0,2.576455,1365000.0,0.000000,43.0,132469.39,1.113383,NaN,1.070325,0.050988,0.002425,15.0,2.0,108.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,1.0,NaN,0.0
2,PNH00002021880,2019,PA,17870,7659.0,1,-0.028803,0.801688,1.602460,4.0,NaN,1.000000,0.792258,0.754876,NaN,0.942981,NaN,1.34,2.65,NaN,0.598457,0.956898,0.43,NaN,NaN,NaN,NaN,0.136978,NaN,0.02,NaN,0.68,0.712508,NaN,NaN,NaN,NaN,NaN,NaN,0.515440,...,0.840372,540.0,0.271822,0.000099,156.0,34.0,1.012262,0.081093,0.986588,0.136689,0.01,299.0,2132.0,0.994657,1144.0,1147.708333,347.1,NaN,34853400.0,420.492525,53.0,89912.10,0.992036,0.592668,0.990512,0.049711,0.001002,84.0,26.0,191.0,0.0,0.0,0.0,NaN,0.0,4.0,0.0,NaN,4.0,0.0
3,BHH10001340552,2016,MA,1342,9704.0,0,0.036574,0.836859,1.363285,8.0,1985.0,NaN,0.754029,0.607456,NaN,0.737216,0.15,1.34,NaN,0.263703,0.639198,NaN,1.79,1.11,0.48,0.37,1900.0,0.932305,1690.5,0.04,0.000002,0.39,0.757101,0.263747,0.08,0.68,0.000004,3381.00,389.29,0.711168,...,0.609092,NaN,NaN,NaN,319.0,74.0,NaN,NaN,NaN,0.183099,0.00,496.0,2354.0,0.561682,1407.0,979.791667,NaN,3.037253,4522100.0,NaN,215.0,140812.14,NaN,NaN,NaN,NaN,0.000202,249.0,81.0,233.0,0.0,NaN,0.0,0.0,0.0,6.0,NaN,1.0,0.0,0.0
4,BHH00001095029,2018,MA,1801,4352.0,0,0.061090,0.841096,0.843938,6.0,NaN,0.888889,0.853551,0.492712,NaN,1.109460,NaN,1.17,NaN,NaN,0.860388,NaN,0.36,1.11,1.04,1.57,NaN,0.844532,2967.0,0.06,NaN,0.29,0.866955,NaN,NaN,1.09,NaN,1483.50,369.60,0.877050,...,1.173017,8465.0,0.927717,0.000097,219.0,29.0,1.004880,0.073291,0.997443,0.239473,0.00,402.0,2247.0,1.186305,2116.0,1336.875000,2929.6,NaN,16772700.0,48.446296,135.0,206702.70,0.998587,NaN,0.994747,0.049255,0.006001,116.0,42.0,199.0,0.0,NaN,0.0,NaN,0.0,6.0,NaN,NaN,4.0,0.0


In [15]:
# create missing value indicator
colist = list(test_df.columns)
df = test_df[colist].isnull().astype(int).add_suffix('_indicator')
cols = [item for x in colist for item in (x, x + '_indicator')]

var = test_df.drop(['pol_id','year','STATE','ZIP5','ZIP4'], axis=1)
var1 = var.apply(lambda x: x.fillna(x.median()))
df1 = pd.concat([var1, df], axis=1).reindex(cols, axis=1)
df1['pol_id'] = test_df['pol_id']
df1['year'] = test_df['year']
df1['STATE'] = test_df['STATE']
df1['ZIP5'] = test_df['ZIP5']
df1 = df1.drop(['pol_id_indicator','year_indicator','STATE_indicator','ZIP5_indicator'], axis=1)
df2 = df1.drop(['ZIP4','ZIP4_indicator','y_indicator'], axis=1)

In [16]:
df2.head()

,pol_id,year,STATE,ZIP5,y,var1222,var1222_indicator,var0989,var0989_indicator,var0202,var0202_indicator,var1165,var1165_indicator,var0063,var0063_indicator,var0797,var0797_indicator,var0967,var0967_indicator,var0403,var0403_indicator,var0222,var0222_indicator,var0424,var0424_indicator,var0371,var0371_indicator,var0484,var0484_indicator,var0547,var0547_indicator,var0065,var0065_indicator,var0975,var0975_indicator,var0457,var0457_indicator,var0233,var0233_indicator,var0260,...,var1257,var1257_indicator,var0525,var0525_indicator,var0624,var0624_indicator,var0775,var0775_indicator,var0636,var0636_indicator,var0640,var0640_indicator,var0992,var0992_indicator,var0867,var0867_indicator,var1234,var1234_indicator,var0921,var0921_indicator,var0350,var0350_indicator,var0535,var0535_indicator,var1146,var1146_indicator,var0100,var0100_indicator,var1145,var1145_indicator,var1171,var1171_indicator,var0505,var0505_indicator,var0123,var0123_indicator,var1173,var1173_indicator,var1204,var1204_indicator
0,PNH00002016383,2019,PA,19380,0,0.006070,0,1.240923,0,0.480738,0,4.0,0,1958.0,0,0.903663,0,1.247416,0,0.485741,0,0.000000,0,1.121520,0,0.71,0,0.64,0,1.19,0,1.308630,0,0.971220,0,1.054577,0,2.38,0,1.01,...,54.0,0,170919.10,0,1.153634,0,0.637127,0,1.083459,0,0.045213,0,0.005820,0,85.0,0,3.0,0,173.0,0,0.0,0,0.0,0,0.0,0,1.0,0,0.0,0,4.0,0,0.0,0,1.0,0,4.0,0,0.0,0
1,PNH00002005641,2018,PA,19149,0,0.022148,0,1.348257,0,0.587569,0,1.0,1,1984.0,0,0.000000,0,1.229200,0,0.634735,0,2.288006,0,0.896893,0,1.05,0,0.46,0,0.00,1,0.606157,0,1.407050,0,1.042516,0,1.63,0,1.01,...,43.0,0,132469.39,0,1.113383,0,0.673835,1,1.070325,0,0.050988,0,0.002425,0,15.0,0,2.0,0,108.0,0,0.0,0,0.0,0,0.0,1,0.0,0,0.0,1,0.0,1,0.0,0,1.0,0,0.0,1,0.0,0
2,PNH00002021880,2019,PA,17870,1,-0.028803,0,0.801688,0,1.602460,0,4.0,0,1963.0,1,1.000000,0,0.792258,0,0.754876,0,1.264126,1,0.942981,0,0.79,1,1.34,0,2.65,0,0.592426,1,0.598457,0,0.956898,0,0.43,0,1.00,...,53.0,0,89912.10,0,0.992036,0,0.592668,0,0.990512,0,0.049711,0,0.001002,0,84.0,0,26.0,0,191.0,0,0.0,0,0.0,0,0.0,0,0.0,1,0.0,0,4.0,0,0.0,0,1.0,1,4.0,0,0.0,0
3,BHH10001340552,2016,MA,1342,0,0.036574,0,0.836859,0,1.363285,0,8.0,0,1985.0,0,0.973653,1,0.754029,0,0.607456,0,1.264126,1,0.737216,0,0.15,0,1.34,0,0.00,1,0.263703,0,0.639198,0,1.001211,1,1.79,0,1.11,...,215.0,0,140812.14,0,1.007463,1,0.673835,1,1.005249,1,0.049072,1,0.000202,0,249.0,0,81.0,0,233.0,0,0.0,0,0.0,1,0.0,0,0.0,0,0.0,0,6.0,0,0.0,1,1.0,0,0.0,0,0.0,0
4,BHH00001095029,2018,MA,1801,0,0.061090,0,0.841096,0,0.843938,0,6.0,0,1963.0,1,0.888889,0,0.853551,0,0.492712,0,1.264126,1,1.109460,0,0.79,1,1.17,0,0.00,1,0.592426,1,0.860388,0,1.001211,1,0.36,0,1.11,...,135.0,0,206702.70,0,0.998587,0,0.673835,1,0.994747,0,0.049255,0,0.006001,0,116.0,0,42.0,0,199.0,0,0.0,0,0.0,1,0.0,0,0.0,1,0.0,0,6.0,0,0.0,1,1.0,1,4.0,0,0.0,0


In [17]:
df2.to_csv('df2.csv')

# start from here

In [3]:
# def reduction(varsource, iter):
#   var650 = pd.read_csv(varsource)
#   # all 650 variables
#   var_list = var650['var'].to_list()
#   var_list.insert(0,'pol_id')
#   id_var.columns = var_list
#   train_df = pd.merge(down_ind, id_var, how='inner', on=['pol_id','year'])


#   count = 1
#   while count < iter:
#     count += 1
#     new_list = var_list[:len(var_list)//2]
#     new_list.insert(0,'pol_id')
#     id_var = id_var[id_var.columns.intersection(new_list)]
#     train_df = pd.merge(down_ind, id_var, how='inner', on=['pol_id','year'])

In [13]:
def reduction(train, test, iter):
  traindf = pd.read_csv(train)
  testdf = pd.read_csv(test)
  # all 650 variables
  var_list = traindf.columns.to_list()

  # reduction
  count = 0
  while count < iter:
    var_list = var_list[:len(var_list)//2]
    traindf = traindf[traindf.columns.intersection(var_list)]
    testdf = testdf[testdf.columns.intersection(var_list)]
    y_train = traindf['y']
    X_train = traindf.drop(['y','ZIP5','pol_id','STATE','year','ZIP4'], axis=1)
    y_test = testdf['y']
    X_test = testdf.drop(['y','ZIP5','pol_id','STATE','year','ZIP4'], axis=1)

    MLA = [
      ensemble.RandomForestClassifier(),
      tree.DecisionTreeClassifier(),
      xgb.XGBClassifier()
      ]

    col = []
    algorithms = pd.DataFrame(columns = col)
    idx = 0

    #Train and score algorithms
    for a in MLA:
      a.fit(X_train, y_train)
      pred = a.predict(X_test)
      pred_train = a.predict(X_train)
      acc = accuracy_score(y_test, pred)
      acc_train = accuracy_score(y_train, pred_train)
      f1 = f1_score(y_test, pred)
      f1_train = f1_score(y_train, pred_train)
      auc = roc_auc_score(y_test, pred)
      auc_train = roc_auc_score(y_train, pred_train)
      recall = recall_score(y_test, pred)
      recall_train = recall_score(y_train, pred_train)
      Alg = a.__class__.__name__
      algorithms.loc[idx, 'Algorithm'] = Alg
      algorithms.loc[idx, 'Accuracy_train'] = acc_train
      algorithms.loc[idx, 'Accuracy_test'] = acc
      algorithms.loc[idx, 'F1 Score_train'] = f1_train
      algorithms.loc[idx, 'F1 Score_test'] = f1
      algorithms.loc[idx, 'AUC Score_train'] = auc_train
      algorithms.loc[idx, 'AUC Score_test'] = auc
      algorithms.loc[idx, 'Recall Score_train'] = recall_train
      algorithms.loc[idx, 'Recall Score'] = recall
      algorithms.loc[idx, 'variables'] = len(var_list)
      idx+=1

    
    algorithms.to_csv(f'id_results_{count}.csv')   
    
    count += 1
    

In [14]:
reduction('/content/drive/MyDrive/BU Capstone/Results/id level saved/down20.csv','/content/drive/MyDrive/BU Capstone/Results/id level saved/in_sample_val.csv',3)